# BREPpy sandbox


**So far:**
- Generate the cell-representing dots
- Implement knn-search
    - (Optimize speed -> Parallelize!)
- Write tests (General correct function, Comparison with scheme-BREP)

**Open questions:**
- How was the spacing 



## Parsing the Parameter file



Not nice: there are about 4 different names for one variable- there are the default ones in the code, the assigned ones in the code, and the ones in the parameter file. 
I will use variables that are similar to the ones defined in the grammar in the code, with the difference that they will be adapted to Python syntax ( _ instead of -), and a few have an additional postfix to clarify what they do (e.g. \_fn = filename) 

### On the installation of neuron
Neuron did not really work out of the box for me. 
I (ubuntu 16.04 LTS, 64 bit) did it the following way:
- Download the .rpm package from here: https://www.neuron.yale.edu/neuron/download
- Install it with: 
    `alien -i nrn_...*package*` (Note that the .deb package did not work out, and neither did the installation using rpm directly)
- Edit the .bashrc file by adding the following lines: 

    `#Added for neuron
    export PYTHONPATH="${PYTHONPATH}:/usr/local/nrn/lib/python/" `
    
    (first check that this path is actually where it got installed by going to the folder and see whether `python -c 'import neuron'` tells you about your NEURON version or whether there ain't no module called neuron.


http://www.davison.webfactional.com/notes/hoc-to-python-bulbnet/

In [1]:
import argparse
import numpy as np
import datetime
import pickle 
import warnings
import neuron
import sys
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#! echo $PYTHONPATH

In [ ]:
%run BREPpy.py

#Todo: Read in from command line for int or bool parameters.
b = Brep()
b.init_from_script(['--config_fn','./input_files/Parameters.hoc'])
b.read_in_config()

## Checking out the output of the original BREP program.

In [2]:
##filenames and input paths for the different files.
import os
from collections import OrderedDict
res_p = os.path.normpath(os.getcwd()+'/output_1/')

fns = OrderedDict ()
fns['aa_gol_dist']='AAtoGoCdistances.dat'
fns['aa_gol_segs']='AAtoGoCsegments.dat'
fns['aa_gol_source']='AAtoGoCsources.dat'
fns['aa_gol_target']='AAtoGoCtargets.dat'

fns['gran_coord'] ='GCcoordinates.sorted.dat'       
fns['gran_t_coord']='GCTcoordinates.sorted.dat'
    
fns['go_coord']='GoCcoordinates.sorted.dat'
fns['go_basd_coord']='GoCbdendcoordinates.sorted.dat'
fns['go_apical_coord']='GoCadendcoordinates.sorted.dat'
fns['go_axon_coord']='GoCaxoncoordinates.sorted.dat'
fns['go_dist']='GoCdistances.dat' 
       
fns['go_go_dist']='GoCtoGoCdistances.dat'
fns['go_go_source']='GoCtoGoCsources.dat'
fns['go_go_targets']='GoCtoGoCtargets.dat'
fns['go_go_gap_dist']='GoCtoGoCgapdistances.dat'
fns['go_go_gap_source']='GoCtoGoCgapsources.dat'
fns['go_go_gap_target']='GoCtoGoCgaptargets.dat'

fns['pf_go_dist']='PFtoGoCdistances.dat'
fns['pf_go_seg']='PFtoGoCsegments.dat'
fns['pf_go_source']='PFtoGoCsources.dat'
fns['pf_go_target']='PFtoGoCtargets.dat'

#for k,v in fns.items():
##    print (k)
#a = import_csv(in_f)
#rr = read_in_coordfile (in_goba)

In [3]:

#def import_csv (fn):
def read_in_coordfile (fn, parse_ignore = True):
    res = []
    with open (fn, newline = '') as f:
        rr = csv.reader(f, delimiter = ' ')
        err = []
        for line in rr:
            ar = []
            for j in range(len(line)):
                try: ar.append(float(line[j]))
                except: err.append(line[j])
            res.append(np.asarray(ar))
    if len(err)> 0 and not parse_ignore: print ('Could not parse on {} instances: {}'.format(len(err), set(err)))
    return np.asarray(res)

#reshape
def coord_reshape (dat):
    dat = dat.reshape([dat.shape[0], int(dat.shape[1]/3),3])
    return dat

#rr = read_in_coordfile(fn)

### Overview over the different output files

In [8]:
# An overview over the different files that BREP produces and the comparison between two different files
for k, v in fns.items():
    print ('Read in file: ', v)
    c_p = os.getcwd()+'/output_1/'+v
    cur = read_in_coordfile(c_p)
    
    c_p2 = os.getcwd()+'/output_2/'+v
    cur2 = read_in_coordfile(c_p2)

    print ('Shape 1 is: ', cur.shape)
    print ('Shape 2 is: ', cur2.shape)
    
    print ('First elements in 1 are:', cur.flatten()[:15])
    print ('First elements in 2 are:', cur2.flatten()[:15])
    
    print (' ')

Read in file:  AAtoGoCdistances.dat
Shape 1 is:  (6710504, 1)
Shape 2 is:  (6791558, 1)
First elements in 1 are: [  72.40834762  134.96968328  134.16163914   57.545658    132.6008968
  131.98243735   54.86484883  131.93994811   54.41139699  131.33764955
   54.84450738   54.73146502  131.55278608   54.29964586   53.71808993]
First elements in 2 are: [ 211.2796311   211.26980386  210.67810779  209.55052216  132.6845687
  209.4256309   132.61236629  132.53076982  208.84039181  132.14997176
  209.00578191   55.24744928  209.00317589  209.14315443  132.33628668]
 
Read in file:  AAtoGoCsegments.dat
Shape 1 is:  (6710504, 2)
Shape 2 is:  (6791558, 2)
First elements in 1 are: [ 1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.]
First elements in 2 are: [ 1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.  3.  1.]
 
Read in file:  AAtoGoCsources.dat
Shape 1 is:  (6710504, 1)
Shape 2 is:  (6791558, 1)
First elements in 1 are: [ 9838.  8704.  8734.  9781.  8713.  8744.  9773.  8526.  97

### Connectivities

In [11]:
gap_t = read_in_coordfile(res_p +'/'+fns['go_go_gap_target'])
pf_t = read_in_coordfile(res_p +'/'+fns['pf_go_target'])
aa_t = read_in_coordfile(res_p +'/'+fns['aa_gol_target'])
gg_t = read_in_coordfile(res_p +'/'+fns['go_go_targets'])


def target_stats (dat, name = ''):
    un, counts = np.unique(dat, return_counts=True)
    print (name)
    print ('Connections per cell: ', round(np.mean(counts),2),'+/-', round(np.std(counts), 2), ', range ', min(counts),'-', max(counts))

target_stats(gg_t, 'Golgi to Golgi:')
target_stats(aa_t, 'AA to Golgi:')
target_stats(pf_t, 'Parallel Fiber to Golgi:')
target_stats(gap_t, 'Golgi to Golgi gap junctions:')

print(
'''
Here are the paper values as a comparison:
Golgi to Golgi:
Connections per cell:  144.85 +/- 36.88 , range  72 - 195
AA to Golgi:
Connections per cell:  554 +/- 302 , range  55 - 1245
Parallel Fiber to Golgi:
Connections per cell:  4759 +/- 1037, range 2512-6582
Golgi to Golgi gap junctions:
Connections per cell:  13.7 +/- 4.6 , range  1-31''')

Golgi to Golgi:
Connections per cell:  144.85 +/- 36.88 , range  72 - 195
AA to Golgi:
Connections per cell:  33552.52 +/- 11268.76 , range  10800 - 54606
Parallel Fiber to Golgi:
Connections per cell:  2.47 +/- 2.25 , range  1 - 10
Golgi to Golgi gap junctions:
Connections per cell:  127.9 +/- 26.65 , range  74 - 166

Here are the paper values as a comparison:
Golgi to Golgi:
Connections per cell:  144.85 +/- 36.88 , range  72 - 195
AA to Golgi:
Connections per cell:  554 +/- 302 , range  55 - 1245
Parallel Fiber to Golgi:
Connections per cell:  4759 +/- 1037, range 2512-6582
Golgi to Golgi gap junctions:
Connections per cell:  13.7 +/- 4.6 , range  1-31


### Golgi-Granule interactions

In [14]:
coord = read_in_coordfile(res_p +'/'+fns['go_coord']) #soma points
pf_t = read_in_coordfile(res_p +'/'+fns['pf_go_target'])
pf_s = read_in_coordfile(res_p +'/'+fns['pf_go_source'])

for s, t in  []

<class 'numpy.ndarray'>


### Golgi-Golgi interactions

In [ ]:
## golgi-golgi interactions
src = read_in_coordfile(res_p +'/'+fns['go_go_sources'])
tar = read_in_coordfile(res_p +'/'+fns['go_go_targets'])
dis = read_in_coordfile(res_p +'/'+fns['go_go_dist'])

res_n = np.zeros(100)
for i in range(100):
    #res_n[i] = np.linalg.norm(coord[int(tar[i]),:]-coord[int(src[i]),:])
    res_n[i] = (sum((coord[int(tar[i]),:]-coord[int(src[i]),:])**2)**0.5)

import networkx as nx
print (res_n)
print (dis[:-100])

# Checking out the connectivities per GC
#n_Go = 200
#source files


### Notable things
- BREP does not seem to take into account the densities from the config file. It just uses the number of Golgi/Granule cells that are defined in its default. The parameters NumGC or NumGoC which is defined in the input dict does not exist in the Parameter.hoc file at all.
- BREP does not seem to check for or find out if there is a number of GCT points that is not related to the number of GC points. It just simulates GC points with aa independently of the GCT points that are the origin of the PF
- The connectivity also does not correspond 

## Golgi cells


### Generation
Shitty here, look at the Check-out-Golgi notebook

In [4]:
def gen_dendrites (som, c_r, c_h, c_m, c_std, c_sp, col = 'kx', plot_fig = False):
    '''Generates dendrites as described in the paper:
    som = coordinates of somata
    c_r = maximal radius of cone
    c_h = height of cone
    c_m = mean angle for each dendrite (number of elements = number of dendrites per cell)
    c_std = standard deviation (degree) for the angle of the dendrite
    c_sp = spacing between the points
    col = color if plot function is enabled
    plot_fig = plot the results?
    Returns a list of lists containing arrays with the coordinates of the dendrites
    '''
    c_n = int(np.linalg.norm([c_r, c_h])/c_sp) #number of points per dendrite
    c_gr = np.linspace(0,1,c_n)*np.ones((3, c_n)) #linspace grid between 0 and 1 with c_n elements
    b_res = []
    for i in range(len(som)): #each cell
        som_c = som[i,:]
        d_res = []
        for cc_m in c_m: #each dendrite
            ep_ang = (np.random.randn()*c_std + cc_m)*np.pi/180 #angle
            pt = ([np.sin(ep_ang)*c_r, np.cos(ep_ang)*c_r, c_h])*c_gr.T #coordinates of the dendrite = endpoint*grid 
            if plot_fig: ax.plot(pt[:,0], pt[:,1], pt[:,2], col);
            d_res.append(pt+som_c) 
        b_res.append(d_res)
    return b_res

a_h = 332.0
a_r= 100.0
b_h = -6.0
b_r = 60.0
a_m = [30.0, 120.0]
b_m = [-20.0, -240.0]
b_std = 10
a_std = 10
a_sp = 6.6
b_sp = 14.4

#coord = read_in_coordfile(res_p +'/'+fns['go_coord']) #soma points

#a_dend = gen_dendrites(coord, a_r, a_h, a_m, a_std, a_sp, 'gx')
#b_dend = gen_dendrites(coord, b_r, b_h, b_m, b_std, b_sp, 'kx')

### Visualization

In [ ]:
# Visualize Golgi cells
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#read in
coord = read_in_coordfile(res_p +'/'+fns['go_coord'])
apical = read_in_coordfile(res_p +'/'+fns['go_apical_coord'])
basal = read_in_coordfile(res_p +'/'+fns['go_basd_coord'])
axon = read_in_coordfile(res_p +'/'+fns['go_axon_coord'])
new_ap = read_in_coordfile('new_apical.dat', parse_ignore=False)
new_bas = read_in_coordfile('new_basal.dat', parse_ignore=False)
new_ax = read_in_coordfile('new_axon.dat', parse_ignore=False)


apical = apical.reshape([apical.shape[0],int(apical.shape[1]/3),3])
basal = basal.reshape([basal.shape[0],int(basal.shape[1]/3),3])
axon = axon.reshape([axon.shape[0],int(axon.shape[1]/3),3])
print (axon.shape)
new_ap = coord_reshape(new_ap)
new_bas = coord_reshape(new_bas)
print( new_ax.shape)
new_ax = coord_reshape(new_ax)


In [ ]:
# dendrite spacing
ap = apical[1,:]
bas = basal[1,:]
resa = [np.linalg.norm(ap[j,:]-ap[j+1,:]) for j in range(len(ap)-1)]
resb = [np.linalg.norm(bas[j,:]-bas[j+1,:]) for j in range(len(bas)-1)]

In [ ]:
# plot
ns = np.arange(1) #neurons to be plotted.

plot_somata = True
plot_apical = 0
plot_basal = 0
plot_axon = 0
plot_new_ap = 0
plot_new_bas  = 0
plot_new_ax = 1
overlay = 0

options= [
    (apical, plot_apical, 'kx'),
    (basal, plot_basal, 'gx'), 
    (axon, plot_axon, 'r.'), 
    (new_ap, plot_new_ap, 'cx'),
    (new_bas, plot_new_bas, 'mx'),
    (new_ax, plot_new_ax, 'y.')]


fig = plt.figure()
ax = fig.gca(projection='3d')
if plot_somata: 
    if not overlay: ax.plot(coord[ns,0], coord[ns,1], coord[ns,2], 'bo')
    else: ax.plot ([0],[0],[0],'bo')

#plot dendrites and axon.
for pts, yn, col in options:
    for i in ns:
        if yn:
            if overlay: pt = pts[i,:,:]-coord[i,:]
            else: pt = pts[i,:,:]
            ax.plot(pt[:,0], pt[:,1], pt[:,2], col)

#ax.view_init(30,180)
proj2D = True
if proj2D:
    plt.figure()
    for pts, yn, col in options:
        for i in ns:
            if yn:
                if overlay: pt = pts[i,:,:]-coord[i,:]
                else: pt = pts[i,:,:]
                plt.plot(pt[:,0], pt[:,1], col)

### Distances

In [ ]:
res = np.zeros((40000, 3))
for i in range(200):
    for j in range(200):
        res[i*200+j,:] = coord[i,:]-coord[j,:]
#dist = [coord[i,:]-coord[j,:] for i,j in range(200)]
dist = read_in_coordfile(res_p +'/'+fns['go_dist'])

print (dist)
print (np.round(-res))
#print (res[1:100,:] + dist[:99,:])
#the distances file just contains all possible differences betwween the different Golgi cells, rounded in an inconsistant way.

### Golgi-Golgi Interactions

In [ ]:

''')

### Connectivities

## Granule cells

In [5]:
aa_length = 200.0
aa_step = 50.0 #this might have to be adapted if the length is not divisable by step
pf_length = 1000.0
pf_step = 7.5


def gen_aa_and_pf (coo, aa_length, aa_step, pf_length, pf_step, plot = False):
    
    if plot:
        fig = plt.figure()
        ax = fig.gca(projection='3d')
    
    aa_nd = int(aa_length/aa_step) + 1 
    aa_sp = np.linspace(0, aa_length, aa_nd)

    pf_nd = int(2*pf_length/pf_step) + 1
    pf_sp = np.linspace(-pf_length, pf_length, pf_nd)

    aa_dots = np.zeros((len(coo), aa_nd, 3))
    pf_dots = np.zeros((len(coo), pf_nd, 3))
    for i, som in enumerate(coo):
        aa_dots[i] = np.ones((aa_nd, 3))*som
        aa_dots[i,:,2] = aa_dots[i,:,2] + aa_sp
        pf_dots[i] = np.ones((pf_nd,3))*aa_dots[i,-1, :]
        pf_dots[i,:,0] = pf_dots[i,:,0] + pf_sp
        if plot:
            plot3d(ax, aa_dots[i], 'ko', markersize=1.0)
            plot3d(ax, pf_dots[i], 'r.', markersize=1.0)
    return aa_dots, pf_dots
#an idea: can we not use a wider, but more random spacing? Like that there are less points, but the connectio


def gen_aa_random (coo, mol_range  = [230, 430]):
    aa_dots = np.array([np.array([coo[i], coo[i]]) for i in range(len(coo))])
    aa_dots[:,1,2] = np.random.uniform(mol_range[0], mol_range[1], len(aa_dots[:,1,2]))
    return aa_dots

def gen_aa_fixed (coo, aa_length =200):
    aa_dots = np.array([np.array([coo[i], coo[i]]) for i in range(len(coo))])
    aa_dots[:,1,2] = aa_dots[:,1,2] + aa_length
    return aa_dots

def gen_pf_from_aa (aa_dots, pf_length):
    ''' '''
    pf_dots = aa_dots.copy()
    pf_dots[:,0,2] = pf_dots[:,1,2] #z axis shall be the same
    pf_dots[:,0,0] = pf_dots[:,0,0] - pf_length
    pf_dots[:,1,0] = pf_dots[:,1,0] + pf_length
    
    return pf_dots







In [ ]:
# 3D Test
coo = Gr_co #[:10,:]
aa_d, pf_d = gen_aa_and_pf (coo, aa_length, aa_step, pf_length, pf_step)

In [ ]:
#2D Test
aa_d2 = gen_aa_random (coo, mol_range  = [230, 430])
aa_d3 = gen_aa_fixed (coo, aa_length =200)

pf_d2 = gen_pf_from_aa (aa_d2, pf_length)

#aa_d2, pf_d2 = gen_pts_for_2dt (coo, aa_length, pf_length)
print (pf_d2.shape)
print (aa_d2.shape)


plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    plt.subplot(1,3,i+1)
    plt.plot(aa_d2 [:,0,j], aa_d2[:,0,k],  'k.', markersize=0.2)
    plt.plot(aa_d2 [:,1,j], aa_d2[:,1,k],  'g.', markersize=0.2)
    plt.plot(pf_d2 [:,1,j], pf_d2[:,1,k],  'r.', markersize=0.2)
    plt.plot(pf_d2 [:,0,j], pf_d2[:,0,k],  'm.', markersize=0.2)
    plt.axis('equal')
    plt.title(tit)

## Population building

In [6]:
# Somata generating method 

#Dimensions of the Granule cell layer
Gc_x = 300
Gc_y = 70
Gc_z = 200

n_Go = int(2000*Gc_x*Gc_y/700/1500)
n_Gr = 2000*n_Go
print (n_Go)
print (n_Gr)

def get_cell_loc (n_cell, Gc_x, Gc_y, Gc_z):
    # get spacing for grid:
    vol_c = Gc_x*Gc_y*Gc_z/n_cell
    sp_def = vol_c**(1/3)/2
    
    #first get a few too many
    gr = np.asarray([[i,j,k] 
                     for i in np.arange(0, Gc_x, 2*sp_def)   
                     for j in np.arange(0, Gc_y, 2*sp_def) 
                     for k in np.arange(0, Gc_z, 2*sp_def)])

    grc = gr + np.random.randn(*gr.shape)*sp_def
    #then remove the ones that lie most outside to get the correct number of cells
    
    lower = grc.T.ravel()
    upper = -(grc-[Gc_x, Gc_y, Gc_z]).T.ravel()
    most_out_idx = np.mod(np.argsort(np.concatenate((lower,upper))), len(grc))
    del_el = len(grc) - n_cell # number of elements to be deleted
    n_del = del_el
    
    while len(np.unique(most_out_idx[:n_del])) < del_el:
        n_del = n_del + del_el - len(np.unique(most_out_idx[:n_del]))
        
    grc = grc[np.setdiff1d(np.arange(len(grc)), most_out_idx[:n_del]),:]
    return grc


def plot3d (ax, dat, *args, **kwargs):
    ax.plot(dat[:,0], dat[:,1], dat[:,2], *args, **kwargs)




40
80000


In [ ]:
# get the two populations
Gr_co = get_cell_loc(n_Gr, Gc_x, Gc_y, Gc_z)
Go_co = get_cell_loc(n_Go, Gc_x, Gc_y, Gc_z)

# 3d plot
fig = plt.figure()
ax = fig.gca(projection='3d')
plot3d(ax, Gr_co, 'k.', markersize=0.2)
plot3d(ax, Go_co, 'ro')

#projections
plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    plt.subplot(1,3,i+1)
    plt.plot(Gr_co[:,j], Gr_co[:,k],  'k.', markersize=0.2)
    plt.plot(Go_co[:,j], Go_co[:,k],  'ro')
    plt.axis('equal')
    plt.title(tit)


## kNN experiments

### Assembling the points

In [7]:
from sklearn.neighbors import KDTree as kdt

#General slice information
Gc_x = 150
Gc_y = 30
Gc_z = 200
#Golgi cell dendrite parameters
a_h = 332.0
a_r= 100.0
b_h = -6.0
b_r = 60.0
a_m = [30.0, 120.0]
b_m = [-20.0, -240.0]
b_std = 10
a_std = 10
a_sp = 6.6
b_sp = 14.4
#Granule cell parameters
aa_length = 200.0
aa_step = 50.0 #this might have to be adapted if the length is not divisable by step
pf_length = 1000.0
pf_step = 7.5

#number of cells (use densities later!)
Go_dens  = 9.5e-6 # from the paper: n_cells/ym³
Gr_Go_ratio = 400


def dend_to_default_format (dend):
    '''change format of the lists from the gen_dendrites function to cells*points*coordinates'''
    dd = np.asarray([np.concatenate((dend[i][0], dend[i][1])) for i in range(len(dend))])
    return dd

# !! Relies on global variables
def full_pop (Gc_x, Gc_y, Gc_z=200, dim2 = True, random_aa = False):
    ''' will give you a full population including points for a given size, in order to make generation easier
    '''
    n_Go = int (Go_dens*Gc_x*Gc_y*Gc_z)
    n_Gr = Gr_Go_ratio*n_Go
    print ('Number of Golgi cells:',n_Go)
    print ('Number of Granule cells:', n_Gr)
    # get the two populations4
    Gr_co = get_cell_loc(n_Gr, Gc_x, Gc_y, Gc_z)
    Go_co = get_cell_loc(n_Go, Gc_x, Gc_y, Gc_z)
    # get the dendrites
    a_dend = dend_to_default_format(gen_dendrites(Go_co, a_r, a_h, a_m, a_std, a_sp, 'gx'))
    b_dend = dend_to_default_format(gen_dendrites(Go_co, b_r, b_h, b_m, b_std, b_sp, 'kx'))
    
    if dim2:
        if random_aa: aa_d = gen_aa_random (Gr_co, mol_range  = [230, 430])
        else: aa_d = gen_aa_fixed (Gr_co, aa_length =200)
        pf_d = gen_pf_from_aa (aa_d, pf_length)
    else: aa_d, pf_d = gen_aa_and_pf(Gr_co, aa_length, aa_step, pf_length, pf_step)
        
    return (Gr_co, Go_co, a_dend, b_dend,  aa_d, pf_d)
    
def flatten_cells (dat):
    return dat.reshape(dat.shape[0]*dat.shape[1],dat.shape[2])

In [ ]:
n_Go = int (Go_dens*Gc_x*Gc_y*Gc_z)
n_Gr = Gr_Go_ratio*n_Go
print (n_Go)
print (n_Gr)

# get the two populations4
Gr_co = get_cell_loc(n_Gr, Gc_x, Gc_y, Gc_z)
Go_co = get_cell_loc(n_Go, Gc_x, Gc_y, Gc_z)

# get the dendrites
a_dend = dend_to_default_format(gen_dendrites(Go_co, a_r, a_h, a_m, a_std, a_sp, 'gx'))
b_dend = dend_to_default_format(gen_dendrites(Go_co, b_r, b_h, b_m, b_std, b_sp, 'kx'))

# get aa and pf
aa_d, pf_d = gen_aa_and_pf(Gr_co, aa_length, aa_step, pf_length, pf_step)

### SKL implementation 1

In [ ]:
Gr_co, Go_co, a_dend, b_dend,  aa_d, pf_d = full_pop (15, 10, Gc_z=200)
kt_aa = kdt(flatten_cells(aa_d))
kt_pf = kdt(flatten_cells(pf_d))
# 4 Golgi cells -> around 40-50 s
# 8 Golgis -> around 130 s -> about 4.3e6 pts --> if we represent all aa with 5 pts, this is feasible (will be around 4e6 pts)

In [ ]:

# connection radius
c_rad_aa = 30.0
c_rad_pf = 5.0
#number of points per granule cell 
n_pt_aa = aa_d.shape[1]
n_pt_pf = pf_d.shape[1]

res_aa_a = []
for i, pt in enumerate(flatten_cells(a_dend)):
    warnings.simplefilter('ignore')
    ind, = kt_aa.query_radius(pt, r= c_rad_aa)
    res_aa_a.append(np.unique((np.floor(ind/n_pt_aa)).astype('int')))
    
res_aa_b = []
for i, pt in enumerate(flatten_cells(b_dend)):
    warnings.simplefilter('ignore')
    ind, = kt_aa.query_radius(pt, r= c_rad_aa)
    res_aa_b.append(np.unique((np.floor(ind/n_pt_aa)).astype('int')))
    

    
res_pf_a = []
gr_f = []
for i, pt in enumerate(flatten_cells(a_dend)):
    warnings.simplefilter('ignore')
    ind, = kt_pf.query_radius(pt, r= c_rad_pf)
    if len(ind) > 0: gr_f.append(i)
    res_pf_a.append(np.unique((np.floor(ind/n_pt_pf)).astype('int')))
    
res_pf_b = []
for i, pt in enumerate(flatten_cells(b_dend)):
    warnings.simplefilter('ignore')
    ind, = kt_pf.query_radius(pt, r= c_rad_pf)
    if len(ind) > 0: print (i)
    res_pf_b.append(np.unique((np.floor(ind/n_pt_pf)).astype('int')))
    

### 2D SKL implementation

**Idea behind this:**
It seems somehow contraintuitive to represent a long line by loads of three-dimensional points.
Instead, it could be much more efficient to represent the line as a dot (if necessary, adjust coordinate system) and do a 2-dimensional nn search. Cut off the Golgi cells that lie outside (In that case, the tree consists of the Golgi dendrites)
This could come in especially handy for the PF. 
The AA are represented by far less points, so in order to make it possible that they might not just be straight lines along the z axis, the tree could be constructed from them (4-5 pts), and then for the Golgi dendrites the search is performed.

**Possible Caveats:**
- The ends are not round
- Small random displacements are not possible
- For a strongly elongated architecture, it is more complicated, as the number of Golgi cells that are considered uselessly increases. 

In [13]:
# Get population
Gr_co, Go_co, a_dend, b_dend,  aa_d, pf_d = full_pop (1500, 700, Gc_z=200, dim2 = True)
#get the right dendrites

# To think about: for the pf only do the search if the points are in the molecular layer.

dends = np.concatenate((flatten_cells(a_dend), flatten_cells(b_dend)))
dends_yz = dends[:,1:]
dends_xy = dends[:,:2]
print ('Size of dendritic tree:', len(dends_xy))

de_tr_yz = kdt(dends_yz)
de_tr_xy = kdt(dends_xy)


c_rad_pf = 5
c_rad_aa = 30

# Golgi indices, apical first
a_ind = (np.ones((a_dend.shape[1], a_dend.shape[0]))*np.arange(a_dend.shape[0])).T
b_ind = (np.ones((b_dend.shape[1], b_dend.shape[0]))*np.arange(b_dend.shape[0])).T
gol_ind = np.concatenate((flatten_cells(np.expand_dims(a_ind, axis = 2)), flatten_cells(np.expand_dims(b_ind, axis = 2))))


Number of Golgi cells: 1995
Number of Granule cells: 798000
Size of dendritic tree: 223440


In [9]:
#Sme playing around on the tree
aa = de_tr_xy.get_arrays()
print (type(aa))
print (len(aa))
print (len(aa[1]))
print (type(aa[1]))
ff = de_tr_xy.node_data
print (ff)
print (len(ff))

<class 'tuple'>
4
21280
<class 'numpy.ndarray'>
<MemoryView of 'array' object>
1023


In [14]:
res_id_pf = []
Gid_pf = dict()

for i in range(len(Go_co)):
    Gid_pf[i] = []

min_z = 200-c_rad_pf # minimal z coordinate to make inquiry reasonable (no parallel fibers possible underneath)
    
print (len(pf_d))
for i, pts in enumerate(pf_d):
    warnings.simplefilter('ignore')
    ind, = de_tr_yz.query_radius(pts[1,1:], r= c_rad_pf)
    ind = ind[np.logical_and(dends[ind,0]<pts[1,0], dends[ind,0]>pts[0,0])]
    gi = (np.unique(gol_ind[ind])).astype('int')
    for k in gi:
        Gid_pf[k].append(i)
    res_id_pf.append(gi)
print (i)
    

#took 10 minutes for whole size (simplest version, no filtering of height and stuff)
    #res_aa_a.append(np.unique((np.floor(ind/n_pt_aa)).astype('int')))

798000
797999


In [15]:
res_id_aa = []
Gid_aa = dict()

for i in range(len(Go_co)):
    Gid_aa[i] = []

min_z = 200-c_rad_aa # minimal z coordinate to make inquiry reasonable (no parallel fibers possible underneath)
    
for i, pts in enumerate(aa_d):
    #if i > 10: break
    warnings.simplefilter('ignore')
    ind, = de_tr_xy.query_radius(pts[1,:2], r= c_rad_aa)
    ind = ind[np.logical_and(dends[ind,2]<pts[1,2], dends[ind,2]>pts[0,2])]
    gi = (np.unique(gol_ind[ind])).astype('int')
    for k in gi:
        Gid_aa[k].append(i)
    res_id_aa.append(gi)
    


**Stats: 28 Golgi cells (150x100)**
- Runtime with just appending Golgi indices to result array: 10.3
- When adding indices to Golgi array: 10.9 -> efficient!

### 2D SKL, the other way around
Idea: Once the tree is constructed, it is fast to search it.
It seems to be better to put the large point cloud in the tree and then query for the points of the small cloud.
With a Granule-Golgi cell ratio of 400:1, putting the granule cells in the tree might be the better option. For the parallel fiber, the amount of points queried for the dendrites can be further decreased by not searching for the ones that are only in the granule cell layer.

In [16]:
pf_tr = kdt(pf_d[:,0,1:])
aa_tr = kdt(aa_d[:,0,:2])
# The goal is to search for the points of the upper part of dend_a in the pf_tr tree.

In [17]:
r_id_pf = []
gr_go_pf = [[] for i in range(len(pf_d))]

min_z = 200-c_rad_pf # minimal z coordinate to make inquiry reasonable (no parallel fibers possible underneath)
    
for i, pt in enumerate(dends):
    if pt[0] > min_z:
        warnings.simplefilter('ignore')
        ind, = pf_tr.query_radius(pt[1:], r= c_rad_pf)
        ind = ind[np.logical_and(pf_d[ind,0,0]<pt[0], pf_d[ind,1,0]>pt[0])]
        for k in ind:
            gr_go_pf[k].append(i)
    else: ind = np.array([])
        #Gid_pf[k].append(i)
    r_id_pf.append(ind)
print (i)

223439


In [18]:
r_id_aa = []
gr_go_aa = [[] for i in range(len(Gr_co))]
go_gr_aa = [[] for i in range(len(Go_co))]

for i, pt in enumerate(dends):
    warnings.simplefilter('ignore')
    ind, = aa_tr.query_radius(pt[:2], r= c_rad_aa)
    ind = ind[np.logical_and(aa_d[ind,0,2]<pt[2], aa_d[ind,1,2]>pt[2])]
    go_gr_aa[gol_ind[i]].append(ind)
    for k in ind:
        gr_go_aa[k].append(i)
    else: ind = np.array([])
        #Gid_pf[k].append(i)
    r_id_pf.append(ind)
print (i)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
tplt = [(), ()]
type(tplt)
type (tplt[0])

### Visualization

In [ ]:
n = 4
# 3d plot
pts_1 = Go_co[n:n+2]
pts_2 = Gr_co[np.array(Gid_aa[n]),:]

fig = plt.figure()
ax = fig.gca(projection='3d')
plot3d(ax, Gr_co, 'k.', markersize=0.2)
plot3d(ax, Go_co, 'ro')
plot3d(ax, pts_1, 'go')
#ax.plot(all_pts[0,0],all_pts[0,1],all_pts[0,2], 'go')
plot3d(ax, pts_2, 'b.', markersize=0.4)
plt.axis('equal')

plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    plt.subplot(1,3,i+1)
    plt.plot(Gr_co[:,j], Gr_co[:,k],  'k.', markersize=0.2)
    plt.plot(Go_co[:,j], Go_co[:,k],  'ro')
    plt.plot(pts_1[:,j], pts_1[:,k], 'go')
    plt.plot(pts_2[:,j], pts_2[:,k], 'b.', markersize=4.0)
    plt.axis('equal')
    plt.title(tit)
    
plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    plt.subplot(1,3,i+1)
    plt.plot(Gr_co[:,j], Gr_co[:,k],  'k.', markersize=0.3)
    plt.plot(Go_co[:,j], Go_co[:,k],  'ro')
    plt.plot(pts_1[:,j], pts_1[:,k], 'go')
    plt.plot(pts_2[:,j], pts_2[:,k], 'b.', markersize=2.0)
    plt.title(tit)


### Scipy implementation

In [ ]:
from scipy.spatial import KDTree as kdt2

aa_d2 = aa_d.reshape(aa_d.shape[0]*aa_d.shape[1], aa_d.shape[2])

def flatten_cells (dat):
    return dat.reshape(dat.shape[0]*dat.shape[1],dat.shape[2])


kt2 = kdt2(aa_d2)
# connection radius
c_rad = 30.0
n_pt_aa = 5

res = []
for i, pt in enumerate(flatten_cells(b_dend)):
    ind = kt2.query_ball_point(pt, r= c_rad)
    ind = np.asarray(ind)
    #print (len(ind[0]))
    #print (len(np.floor(ind[0]/n_pt_aa)))
    res.append((np.floor(ind/n_pt_aa)).astype('int'))

In [ ]:
### deprecated!

all_pts = np.zeros((1,3))
pt_order = [a_dend, b_dend, aa_d]
start_ind = np.zeros(len(pt_order))
pts_per_cell = np.zeros(len(pt_order))

for i, pts in enumerate(pt_order):    
    pts_f = pts.reshape(pts.shape[0]*pts.shape[1], pts.shape[2])
    print (pts_f.shape)
    start_ind[i] = len(all_pts)-1
    pts_per_cell[i] = pts.shape[1]
    all_pts = np.concatenate((all_pts, pts_f))
all_pts = all_pts[1:,:]

print (start_ind)
print (pts_per_cell)

aa_d2 = aa_d.reshape(aa_d.shape[0]*aa_d.shape[1], aa_d.shape[2])

kt = kdt(aa_d2)
# connection radius
c_rad = 30.0
# The index at which the source ids start
so_ind_start = int (start_ind[2])
# source cells: number of points per cell
so_ppc = pts_per_cell [2]


## Kd-Trees

### General Background

- binary search stree
    - every branching node contains a k-dimensional point
    - every leaf node contains a set of points
- every branching node represents a splitting hyperplane that divides the space into two half-spaces    
    - left of the splitting hyperplane = left subtree, same for richt
    - each spliitting hyperplane is perpendicular to one of the axes in the k-dimensional space
    - the axes for the splitting hyperplanes are rotating

### On the chicken kd-tree library:
- works with datastructure POINT3D
    - constructor: make-point3d dbl dbl dbl 
    - accessors point3d-x /y/z
    - predicate: point3d?
- KD-Tree itself
    - constructor: list->kd-tree (list of POINT3D)
    - predicates: kd-tree? -> checks object, kd-tree-empty?, 
        - kd-tree-is-valid? -> checks if all points in subtree lie on left side of hyperplane and right on right
        - kd-tree-all-subtrees-are-valid? -> valid property for all branching nodes?
    - accessors: 
        - kd-tree->list  -> all the points contained in tree in a POINT3D list
        - kd-tree->list\* -> list with elements of the form (i . POINT3D) -> i is the relative integer index of the point
     
for other accessors, the author was not motivated enough to write a description.


### Pythonic Kd-Tree libraries:

#### Scipy:
- Documentation: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html
- Source Code: https://github.com/scipy/scipy/blob/master/scipy/spatial/kdtree.py
- Algorithm reference Maneewongvatana and Mount 1999
- Can be queried for r nearest neighbors, however r should be relatively small because elsewise, brute force is just as efficient.
- Approximate nearest neighbors seems to be another, and much faster option, and might work well for us.
- Uses pythonic libraries. 
- The heap queue algorithm: https://github.com/python/cpython/blob/2.7/Lib/heapq.py seems to be used, but it is also in python


#### SKlearn:
- Documentation: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html
- Source Code: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/neighbors/kd_tree.pyx
- uses cython. Might thus be faster. Let's check.


Neither of them appears to have parallelization supported right from the beginning.
Both just take regular arrays as inputs.


Could be interesting:
http://ieeexplore.ieee.org/abstract/document/5654017/?reload=true
GPU implementation for kNN search
Following this:
https://link.springer.com/chapter/10.1007/978-3-642-38628-2_67



In [ ]:
from scipy.spatial import KDTree as T_sp
from sklearn.neighbors import KDTree as T_sk
s_dat = np.asarray(import_csv(fn_out))
import time

rn = np.random.randint(0, len(s_dat), 50)
nn = 10

In [ ]:
print (s_dat.shape)

In [ ]:
k_dist = np.zeros((len(rn), nn))
k_ind = np.zeros((len(rn), nn))
tk0 = time.time()
kt = T_sk(s_dat)
tk1 = time.time()
for i, ii in enumerate(rn):
    warnings.simplefilter('ignore') #definitely not ideal. But no clue how the validation file gets called
    k_dist[i,:], k_ind [i,:] = kt.query(s_dat[ii,:], k = nn)
tk2 = time.time()

In [ ]:
p_dist = np.zeros((len(rn), nn))
p_ind = np.zeros((len(rn), nn))
tp0 = time.time()
pt = T_sp(s_dat)
tp1 = time.time()
for i, ii in enumerate(rn):
    p_dist[i,:], p_ind [i,:] = pt.query(s_dat[ii,:], k = nn)
tp2 = time.time()

In [ ]:
#Here is the first, blunt comparison of both algorithms in terms of time:
print (tk1-tk0)
print (tp1-tp0)

print (tk2-tk1)
print (tp2-tp1)
#Scikit-learn is way faster.

#print (np.isclose (k_dist, p_dist))
#print (k_ind -p_ind) 
# Where the indices are not the same it is because the distances are equal.

In [ ]:
# Reading in, subsampling and storing again the csv files in order to have a smaller dataset at hand that has a similar density anyway.

import csv

fn_in = 'input_files/GCTcoordinates.dat'
fn_out = 'input_files/GCT_small.dat' #100x150 -> 11 k
fn_out2 = 'input_files/GCT_smallsmall.dat' #30x100 -> 2.2k
fn_out3 = 'input_files/GCT_tiny.dat' # 2x5 ->  6

x_r = [0.0, 2.0]
y_r = [0.0, 5.0]
z_r = [0.0, 1000.0]
rrs = [x_r, y_r, z_r]

def subsample_coords (rrs, fn_in, fn_out = 'input_files/downsampled.dat', save = True):
    res = []
    rnr = [0, 0]
    with open(fn_in, newline = '') as f, open (fn_out, 'w', newline = '') as w_f:
        rr = csv.reader(f, delimiter = ' ')
        if save: wr = csv.writer(w_f, delimiter = ' ')
        for line in rr:
            in_range = all([float(line[i])>rrs[i][0] and float(line[i])<rrs[i][1] for i in range(len(rrs))]) #check if in range
            if in_range: 
                if save: wr.writerow([float(line[j]) for j in range(len(rrs))])
                res.append([float(line[j]) for j in range(len(rrs))])
                rnr[0] = rnr[0]+1
            else:
                rnr[1] = rnr[1]+1
    print ('Subsampled {} of {}'.format(rnr[0], rnr[1]))
    return res

#my_s = subsample_coords (rrs, fn_in, fn_out3, save = True)

def import_csv (fn):
    res = []
    with open (fn, newline = '') as f:
        rr = csv.reader(f, delimiter = ' ')
        for line in rr:
            res.append([float(line[j]) for j in range(len(line))])
    return np.asarray(res)


## Random and deprecated stuff below this.

In [ ]:
# Development site for the read_in_config function.

#from neuron import hoc, h
# weird thing that I did not get yet: despite all the copy statements, the second time you calculate d_l, it would give 0. 
# Thus, the hoc objects must somehow take on each other's parameters... 

#load an empty hoc object and find out which parameters are native to that object (probably useless...)
empty_hoc = dir(neuron.hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
overwrite_config =  True
#load our own hoc object from the parameter file, get the disjunct list of parameters (probably useless...)
neuron.h.xopen(config_fn)
full_hoc = dir(neuron.h)
if 'd_l' not in globals():
    d_l = list (set (full_hoc)  - set (empty_hoc)).copy()

#c_d = b.config_dict
c_d = dict((v,k) for k,v in b.config_dict.items()) #exchange key and value
#this dict translates the parameters used in the Parameters file to the ones used in the code
# Check if the Brep object contains the right parameters and if so, change them.
# Note: Resolve conflicts with the command line - I think default should be that command line should has priority 
self = b
for h_k in full_hoc:
    if h_k in c_d.keys() and h_k not in self.cl_args.keys():
        if hasattr (self.args, c_d[h_k]):
            setattr (self.args, c_d[h_k], getattr (neuron.h, h_k))
        else:
            print ('Did not find {}'.format(c_d[h_k]))
    elif h_k in c_d.keys() and h_k in self.cl_args.keys():
        if hasattr (self.args, c_d[h_k]):
            if overwrite_config:
                warnings.warn('Parameter {} was set both by command line and in config, will use value from command line'.format(c_d[h_k]))
            else:
                warnings.warn('Parameter {} was set both by command line and in config, will use value from config file'.format(c_d[h_k]))
                setattr (self.args, c_d[h_k], getattr (neuron.h, h_k))
    
# The following two parameters are an exception:
if 'GLdepth' in d_l and 'PCLdepth' in d_l and not 'aa-length' in self.cl_args.keys():
    setattr (self.args, 'aa-length', getattr(neuron.h, 'GLdepth')+getattr(neuron.h,'PCLdepth'))
    


In [ ]:
### The transformations of the parameters, automized in a small parser script.

#Had to be done only once, but will be kept for reference.

from neuron import hoc, h
# weird thing that I did not get yet: despite all the copy statements, the second time you calculate d_l, it would give 0. 
# Thus, the hoc objects must somehow take on each other's parameters... 

#load an empty hoc object and find out which parameters are native to that object (probably useless...)
empty_hoc = dir(hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
#load our own hoc object from the parameter file, get the disjunct list of parameters (probably useless...)
h.xopen(config_fn)
full_hoc = dir(h).copy()
if 'd_l' not in globals():
    d_l = list (set (full_hoc)  - set (empty_hoc)).copy()

# Code file
tf = 'brep_commented.scm'
# Step one: parse all lines that contain both config or options as those are the ones that 
res_dict = {}
with open (tf, 'rb') as tff:
    for line in tff:
        st = str(line)
        c = st.find ("config '")
        o = st.find ("options '") 
        if c > 0 and o > 0:
            c_clb = st[c:].find (')')
            o_clb = st[o:].find (')')
            res_dict[st[c+8:c+c_clb]] = st[o+9:o+o_clb] 
    tff.close()

#parameters that are defined in the res file but have not been parsed yet
rest_hk = (set (d_l)- set(res_dict.keys()))
rem = {}
#check if they occur in the code
with open ('brep_commented.scm', 'rb') as f_in:
    n = 0
    for line in f_in:
        n = n+1
        for w in rest_hk:
            if str(line).find(w)>0:
                if w in rem.keys():
                    rem[w].append(n)
                else:
                    rem[w] = [n]                  
print (rem) # 'TS is coincidental, the other two parameters are taken seperate care of.

#print the resulting dict to a file.
with open ('par_d2.txt', 'w') as f_out:
    for k in res_dict.keys():
        f_out.write("'" +res_dict[k]+ "' : '"+k+ "', \n" )

#print (res_dict)

In [ ]:
#Used this to try out command line calls. 

#Brep = importlib.reload(BREPpy)
#stupid workaround so that the known command line call can be kept up.
#I think I changed something, would have to git checkout....
class Brep2 (Brep, b):
    def __init__:
        self.args = b.args
        self.config_dict = b.config_dict
        self.cl_args = b.cl_args
        


def new_Brep (arg_dict = {}, **kwargs):
    if True: #delete and make new Brep file
        ! python ~/Desktop/LabRot_OIST/pybrep/BREPpy.py --config_file blabla 
        a = pkl.load(open('./tmp.pkl', 'rb'))
        ! rm tmp.pkl
        b = Brep2(a)
    else: b = Brep2(pkl.load(open('./tmp.pkl', 'rb'))) 
    #Process and add arguments    
    arg_dict.update(kwargs)
    for k in arg_dict.keys():
        if hasattr (b.args, k):
            setattr (b.args, k, arg_dict[k])
        else:
            warnings.warn ('Keyline argument {} not known'.format(k))
            
    return b

arg_dict = {'config_file': 'blabla.c',
            'verbose': True}
b = new_Brep(arg_dict, gc_points_fn = 'yipyip' )


In [ ]:
## Fun with magic

# http://ipython.readthedocs.io/en/stable/interactive/magics.html
#notable ones
# %debug #-> debug stuff. Lets you inspect the stack frame of an exception interactively
# %env #-> see all env variables

In [ ]:
### Used this to develop the soma rendering algorithm

dat = Gr_co

print (dat.shape)

lower = dat.T.ravel()
upper = -(dat-[Gc_x, Gc_y, Gc_z]).T.ravel()
most_out_idx = np.mod(np.argsort(np.concatenate((lower,upper))), len(dat))

n_del = 500
dat = dat[np.setdiff1d(np.arange(len(dat)), most_out_idx[:n_del]),:]

print (dat.shape)

test_ar = np.array([[0,1,3],[-1,3,5],[2,3,4], [5,3,2]])
print (test_ar.ravel())
print (np.argsort(test_ar.T.ravel()))
tt2 = test_ar- [1,2,3]
print (tt2)

print ('')
most_out = np.argsort(np.concatenate(( test_ar.T.ravel(), -(test_ar-[1,2,3]).T.ravel() )))

most_out_idx = np.mod(most_out, len(test_ar))

print (len(test_ar))
print( -(test_ar-[1,2,3]))
print( -(test_ar-[1,2,3]).ravel())
print (np.argsort( -(test_ar-[1,2,3]).ravel()))

print ('')
print (np.concatenate(( test_ar.ravel(), -(test_ar-[1,2,3]).ravel() )))
print (most_out)
print (most_out_idx)
print 

print ('')
print (test_ar)
print( -(test_ar-[1,2,3]))
#hs = np.sum(np.heaviside(-dat) + np.heaviside(dat-[Gc_x, Gx_y, Gc_z]))

In [ ]:
(332**2 + 100**2)**0.5 / 24

In [ ]:
(6**2+60**2)**0.5/12